In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from substrate_helpers import *

/home/chia/miniconda3/lib/python3.8/site-packages/ipfshttpclient/client/__init__.py:73: VersionMismatch: Unsupported daemon version '0.13.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


In [73]:
import sqlite3
import re
import datetime
import pandas as pd

In [4]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

substrate = SubstrateInterface(
    url="ws://127.0.0.1:11946",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [138]:
con = sqlite3.connect('test1.db')
cur = con.cursor()

In [137]:
con.close()

In [32]:
post_schemas = {}
for i in [post_schemaId, comment_schemaId, vote_schemaId, user_schemaId]:
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    print(schemaTemp.value)
    post_schemas[schemaTemp.value] = i

category STRING,username STRING,profile_pic STRING,title STRING,body STRING,url STRING,is_nsfw NUMERIC
post_hash STRING,parent_hash STRING,parent_type STRING,username STRING,profile_pic STRING,body STRING
post_hash STRING,parent_hash STRING,parent_type STRING,num_votes NUMERIC
msa_id NUMERIC,username STRING,password STRING,profile_pic STRING,wallet_ss58_address STRING


In [196]:
def update_db(start_block=0, backfill=True):
    current_block = substrate.get_block()['header']['number']
    schemas = {"post": post_schemaId, "comment": comment_schemaId, "vote": vote_schemaId, "user": user_schemaId}
    date_format = "%m-%d-%Y %H:%M:%S"

    for schemaName, schemaId in schemas.items():
        print(schemaName)
        schemaValue = substrate.query(
            module='Schemas',
            storage_function='Schemas',
            params=[schemaId]
        ).value

        extraValues = "block_number INTEGER,msa_id_from_query INTEGER,signer STRING,date_minted DATE"
        is_ipfs_hash = schemaName not in ['vote', 'user']
        if is_ipfs_hash:
            extraValues += ",ipfs_hash STRING"

        if backfill:
            # Delete table if exists and then create it
            cur.execute(f"DROP TABLE IF EXISTS {schemaName}")
            
            names = ','.join([v.split(' ')[0] for v in schemaValue.split(',') + extraValues.split(',')])
            create_table = f"CREATE TABLE {schemaName} ({schemaValue}, {extraValues}, UNIQUE({names}))"
            print(create_table)
            cur.execute(create_table)

        params = [
            schemaId,
            {
                "page_size": 1000,
                "from_block": start_block,
                "to_block": current_block,
                "from_index": 0,
            }
        ]

        request = substrate.rpc_request(
            method='messages_getBySchema',
            params=params,
        )

        for content in request['result']['content']:
            date_str = "null"
            for extrinsic in substrate.get_block(substrate.get_block_hash(content['block_number']))['extrinsics']:
                if "Timestamp" == extrinsic.value['call']['call_module']:
                    timestamp = extrinsic.value['call']['call_args'][0]['value']
                    date_time = datetime.datetime.fromtimestamp(timestamp/1000)
                    date_str = "'" + date_time.strftime(date_format) + "'"
                    break
            if date_str is None:
                print('Failed to get timestamp from block ', )

            row_raw = bytes.fromhex(content['data'][2:]).decode()
            ipfs_hash = None
            if is_ipfs_hash:
                ipfs_hash = row_raw
                try:
                    row_raw = client.cat(ipfs_hash).decode()
                except:
                    print(ipfs_hash)
                    continue
            try:
                row = json.loads(row_raw)
            except:
                continue

            inp = ""
            for scheme in schemaValue.split(','):
                scheme_list = scheme.split(' ')
                data = row[scheme_list[0]]
                data_type = scheme_list[1]
                if 'string' in data_type.lower():
                    data = data.replace("'", "❜")
                    inp += f"'{data}',"
                else:
                    inp += f"{data},"

            inp += f"{content['block_number']},{content['msa_id']},'{content['signer']}',{date_str}"
            if is_ipfs_hash:
                inp += f",'{ipfs_hash}'"

            cur.execute(f"INSERT OR IGNORE INTO {schemaName} VALUES ({inp})")
    return current_block

In [197]:
last_block_updated = update_db(start_block=0, backfill=True)
con.commit()

post
CREATE TABLE post (category STRING,username STRING,profile_pic STRING,title STRING,body STRING,url STRING,is_nsfw NUMERIC, block_number INTEGER,msa_id_from_query INTEGER,signer STRING,date_minted DATE,ipfs_hash STRING, UNIQUE(category,username,profile_pic,title,body,url,is_nsfw,block_number,msa_id_from_query,signer,date_minted,ipfs_hash))
,,post,,QmfGhEYRYZoC3Vm6ijmdPdjAU6ZXMJmuhrAhpChvDBcH6p
comment
CREATE TABLE comment (post_hash STRING,parent_hash STRING,parent_type STRING,username STRING,profile_pic STRING,body STRING, block_number INTEGER,msa_id_from_query INTEGER,signer STRING,date_minted DATE,ipfs_hash STRING, UNIQUE(post_hash,parent_hash,parent_type,username,profile_pic,body,block_number,msa_id_from_query,signer,date_minted,ipfs_hash))
vote
CREATE TABLE vote (post_hash STRING,parent_hash STRING,parent_type STRING,num_votes NUMERIC, block_number INTEGER,msa_id_from_query INTEGER,signer STRING,date_minted DATE, UNIQUE(post_hash,parent_hash,parent_type,num_votes,block_number,

In [152]:
query = '''
SELECT * FROM user
'''
df = pd.read_sql_query(query, con)
df

,msa_id,username,password,profile_pic,wallet_ss58_address,block_number,msa_id_from_query,signer,date_minted
0,438,Mental_Shelter,password,https://www.redditstatic.com/avatars/defaults/...,5DUemZiN3jQKfaedGo1bEQKRnercMrgAHianX21hkrcChEuC,7397,438,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:36:24
1,439,dimechimes,password,https://www.redditstatic.com/avatars/defaults/...,5E741P4DGM94ozrLjrLpNJe9HGPgT5TCAQs1yPbYvbALRDMZ,7398,439,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:36:54
2,440,Sl0ppy0tter,password,https://styles.redditmedia.com/t5_52wkdg/style...,5EFK76PX3ho6qGVYax1z3mEyHYxYKJxpAE4M1mV12NgSPAXn,7399,440,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:37:12
3,441,mightylordredbeard,password,https://www.redditstatic.com/avatars/defaults/...,5GKV2pAsjGJUKJ9wEW4tF4yxL2ogtUvox8Gyejdd4wHa5VAi,7400,441,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:37:36
4,442,human_male_123,password,https://www.redditstatic.com/avatars/defaults/...,5GBg27doAGAJasPjcbNvrbGrMDvkYBxM2CY2CFQACye7UGr7,7401,442,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 19:38:00
...,...,...,...,...,...,...,...,...,...
302,740,brigrrrl,password,https://styles.redditmedia.com/t5_28rzsh/style...,5CotCQUjesc6BnWt6x7YyersypyaCuswKoVPqXsjDXtc3wH3,7722,740,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:46:24
303,741,eduardtee,password,https://styles.redditmedia.com/t5_6msk8/styles...,5CLxfPDYkXFUnGYzJFPkQiKdXVetR81UcxqnfygmwpwtFJuG,7723,741,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:46:54
304,742,Victory1871,password,https://www.redditstatic.com/avatars/defaults/...,5EFmBd3PDykbXC1YfQXxGbndgR9nJubokV3BENTB6TXva4Hx,7725,742,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:47:36
305,743,Stringbean1109,password,https://styles.redditmedia.com/t5_5uhgku/style...,5GmuriRZFWdMhQhU1mN8t1nzMvcKcvWd5B9bKzTVFz1zLooi,7726,743,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:48:00


In [156]:
import uvicorn
uvicorn.server.Config?

In [153]:
query = '''
SELECT * FROM comment
'''
df = pd.read_sql_query(query, con)
df

,post_hash,parent_hash,parent_type,username,profile_pic,body,block_number,msa_id_from_query,signer,date_minted,ipfs_hash
0,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,MedicatedAxeBot,https://www.redditstatic.com/avatars/defaults/...,Dank[.](https://i.imgur.com/3bQtuMO.png)\n\n--...,7118,189,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:44:54,QmSUp3SQeLvojwNfxYFj6DUDZCo44iKDfwJT3xyHGEzkFh
1,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,Vendis123,https://styles.redditmedia.com/t5_qr7mm/styles...,"Well, he doesn❜t write his own speeches",7119,190,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:45:12,QmdGPH6RK7Pj31NEtkrECKzMvQBksYyEiv9XQGc7K2dFHX
2,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,Dubr1s,https://styles.redditmedia.com/t5_1nasz1/style...,And it❜s most upvoted comment with a lot of aw...,7120,191,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:45:36,QmfDvg9i9RqbPCPemgnDdPR6HsG2YFeY4dZinR9QVQ3iky
3,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,iron_fan_man,https://styles.redditmedia.com/t5_38wc0r/style...,YES he’s like bob the banana,7121,192,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:46:00,QmeJxRNR92e7DVpEpZKfstRnkDCkjywGsfrPvuZPcBKe9D
4,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,QmX2Ny3mMA4pLZiW2XnrNTFda7iHgQtgq46TbaPmpXF25G,post,Probablyaretweetbot,https://styles.redditmedia.com/t5_4rw7la/style...,u/deleted yo,7122,193,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 17:46:24,QmZTmoZSaCypPbXkfDYdHfSA2UAnVPBhzx2VAYD5e3cAFB
...,...,...,...,...,...,...,...,...,...,...,...
527,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,post,brigrrrl,https://styles.redditmedia.com/t5_28rzsh/style...,I love this. Reminds me of my childhood a bit....,7722,740,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:46:24,QmNWqPDQ58GuazdSLyuaCV7E3w7hDgvrcBUXunKUytt154
528,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,post,eduardtee,https://styles.redditmedia.com/t5_6msk8/styles...,So sweet. Your parents deserve a hug for this ...,7723,741,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:46:54,QmdV5r1NDRwGe5ZieEEtSVhJtDETXwQ22ZD3pAxEafvVYz
529,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,post,unclejarjarbinks,https://styles.redditmedia.com/t5_180224/style...,Best part was you didn❜t have to deal with the...,7724,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:47:12,QmW8Zg4cqFGH9XjLYTL3er6W4VCBw2RRxmCaRP8SFsU49n
530,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,QmfGyt1GNdPBy5b1WbXGi6PBtsSmtDwtGiNjXg6BEXXXGh,post,Victory1871,https://www.redditstatic.com/avatars/defaults/...,Wow that’s wholesome. This just made my day.,7725,742,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 21:47:36,QmX9XL7kS92WfgV9AjYDX11BaWakLpqr8FGqpazRQyL6s8


In [87]:
date_time = datetime.datetime.now() - datetime.timedelta(minutes=22)
date_str = "'" + date_time.strftime(date_format) + "'"
date_str

"'06-21-2022 17:55:21'"

In [88]:
print(date_str)

'06-21-2022 17:55:21'


In [98]:
lower_bound = 1
upper_bound = 11
time_filter = {'minutes_15': 15, 'hour': 60, 'day': 60*24, 'week':60*24*7, 'month':60*24*30}
date_time = datetime.datetime.now() - datetime.timedelta(minutes=time_filter['minutes_15'])
date_str = "'" + date_time.strftime(date_format) + "'"

query = f'''
SELECT * FROM post 
JOIN (
    SELECT parent_hash, SUM(num_votes) AS total_votes 
    FROM vote GROUP BY parent_hash
) votes ON votes.parent_hash = post.ipfs_hash
WHERE post.date_minted >= {date_str}
ORDER BY total_votes DESC
LIMIT {lower_bound}, {upper_bound}
'''
df = pd.read_sql_query(query, con)
df

,category,username,profile_pic,title,body,url,is_nsfw,block_number,msa_id_from_query,signer,date_minted,ipfs_hash,parent_hash,total_votes
0,interestingasfuck,Real_Prophet01,https://styles.redditmedia.com/t5_6dmhau/style...,Zero to hero before & after animation,,https://v.redd.it/jlmw0wvqsw691,0,7183,1,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 18:10:54,QmWJWbHKkKBfPSbuTWXbNQFojSRUuSRojwnPpKJzaCYrqM,QmWJWbHKkKBfPSbuTWXbNQFojSRUuSRojwnPpKJzaCYrqM,61074
1,interestingasfuck,a7kilr,https://styles.redditmedia.com/t5_2d6p3a/style...,Cloudflare has a wall full of lava lamps they ...,,https://i.redd.it/is89fqceu0791.jpg,0,7194,262,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,06-21-2022 18:15:12,QmfGoFEXq1Meqpz3uE17qwCsGdHQKyeoW4RS3rHsEJTqMh,QmfGoFEXq1Meqpz3uE17qwCsGdHQKyeoW4RS3rHsEJTqMh,60767


In [99]:
df.to_json()

'{"category":{"0":"interestingasfuck","1":"interestingasfuck"},"username":{"0":"Real_Prophet01","1":"a7kilr"},"profile_pic":{"0":"https:\\/\\/styles.redditmedia.com\\/t5_6dmhau\\/styles\\/profileIcon_snooc3085ebd-caa2-4922-ac5a-7e7ab0e39c43-headshot.png?width=256&height=256&crop=256:256,smart&s=0e768c793c8b5721e7f1a0fd0b5714f11ca001f9","1":"https:\\/\\/styles.redditmedia.com\\/t5_2d6p3a\\/styles\\/profileIcon_pgoq4kgdlza81.jpg?width=256&height=256&crop=256:256,smart&s=de9ddb02fbfedc1850f6fb41585b40629412759c"},"title":{"0":"Zero to hero before & after animation","1":"Cloudflare has a wall full of lava lamps they feed into a camera as a way to generate randomness to create cryptographic keys"},"body":{"0":"","1":""},"url":{"0":"https:\\/\\/v.redd.it\\/jlmw0wvqsw691","1":"https:\\/\\/i.redd.it\\/is89fqceu0791.jpg"},"is_nsfw":{"0":0,"1":0},"block_number":{"0":7183,"1":7194},"msa_id_from_query":{"0":1,"1":262},"signer":{"0":"5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty","1":"5FHneW46xG

In [129]:
        post_hash = "QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT"
        query = f'''
        SELECT * FROM post 
        JOIN (
            SELECT parent_hash, SUM(num_votes) AS total_votes 
            FROM vote GROUP BY parent_hash
        ) votes ON votes.parent_hash = post.ipfs_hash
        WHERE post.ipfs_hash = '{post_hash}'
        ORDER BY post.date_minted DESC
        '''
        df = pd.read_sql_query(query, con)

In [142]:
        post_hash = "QmaEKD9igfwiDNMVufvXTSQ3SDorSna1fXBzhpquYGdZwT"
        query = f'''
        SELECT * FROM user
        '''
        df = pd.read_sql_query(query, con)
        df

,msa_id,username,password,profile_pic,wallet_ss58_address,block_number,msa_id_from_query,signer,date_minted


In [131]:
for row in cur.execute("PRAGMA table_info('post')"):
    print(row)

(0, 'category', 'STRING', 0, None, 0)
(1, 'username', 'STRING', 0, None, 0)
(2, 'profile_pic', 'STRING', 0, None, 0)
(3, 'title', 'STRING', 0, None, 0)
(4, 'body', 'STRING', 0, None, 0)
(5, 'url', 'STRING', 0, None, 0)
(6, 'is_nsfw', 'NUMERIC', 0, None, 0)
(7, 'block_number', 'INTEGER', 0, None, 0)
(8, 'msa_id_from_query', 'INTEGER', 0, None, 0)
(9, 'signer', 'STRING', 0, None, 0)
(10, 'date_minted', 'DATE', 0, None, 0)
(11, 'ipfs_hash', 'STRING', 0, None, 0)


In [ ]:
for schema, contents in content_jsons.items():
    schema_items = schema.split(',')
    
    for content in contents:
        ipfs_hash = bytes.fromhex(content['data'][2:]).decode()
            
        data = json.loads(client.cat(ipfs_hash).decode())

In [198]:
def subscription_handler(obj, update_nr, subscription_id):
    last_block_updated = update_db(start_block=obj['header']['number'], backfill=False)
    con.commit()
    print(last_block_updated)


result = substrate.subscribe_block_headers(subscription_handler, include_author=False)

post


SubstrateRequestException: {'code': 100, 'message': 'Dispatch Error Module:35 error:2', 'data': ''}

In [164]:
def subscription_handler(obj, update_nr, subscription_id):
    return obj, update_nr, subscription_id
result = substrate.subscribe_block_headers(subscription_handler, include_author=False)

In [211]:
e = substrate.get_events(substrate.get_block_hash(result[0]['header']['number']))

In [218]:
substrate.

'0x36fa31a08a929dd9f08cd57a678cbb877f9d55280db90a727a6d56a14971796f'

In [214]:
e[1].value

{'phase': 'Initialization',
 'extrinsic_idx': None,
 'event': {'event_index': '2300',
  'module_id': 'Messages',
  'event_id': 'MessagesStored',
  'attributes': (159, 8033, 2)},
 'event_index': 35,
 'module_id': 'Messages',
 'event_id': 'MessagesStored',
 'attributes': (159, 8033, 2),
 'topics': []}

In [199]:
result[0]

{'header': {'digest': {'logs': [<scale_info::13(value={'PreRuntime': ('0x61757261', '0xaa8f390800000000')})>,
    <scale_info::13(value={'Seal': ('0x61757261', '0x306ac3ca56ae8e2d4db203c1ae908045547dc039946de605a83d62495ad39f7d6e1eb228029d05b6bccc85d2889fa93b7d2185bb7ffa2dda1f23fe64000a1583')})>]},
  'extrinsicsRoot': '0xe8fec3d67c271817e646e4aab48625c363d84a28317a5a1c5fb432f6e70d541b',
  'number': 8034,
  'parentHash': '0x6f5ac10fd71100e7aadbc3900569a04b9027554370837e130dad29e1bd825751',
  'stateRoot': '0xa8ea03e6d61a4584345f2c12236b4a001f2e7cdc59fb70761fb6f5eb2c659de6'}}

In [203]:
result[0]['header']['digest']['logs'][0]

<scale_info::13(value={'PreRuntime': ('0x61757261', '0xaa8f390800000000')})>

In [31]:
date_str = None
for extrinsic in substrate.get_block(substrate.get_block_hash(2))['extrinsics']:
    if "Timestamp" == extrinsic.value['call']['call_module']:
        timestamp = extrinsic.value['call']['call_args'][0]['value']
        date_time = datetime.fromtimestamp(timestamp/1000)
        date_str = date_time.strftime("%m/%d/%Y, %H:%M:%S")
        break
if date_str is None:
    print('Fail')
date_str

'06/19/2022, 18:16:24'

In [24]:
extrinsic = substrate.get_block(substrate.get_block_hash(2))['extrinsics'][1]

In [26]:
extrinsic.value['call']

{'call_index': '0x0200',
 'call_function': 'set',
 'call_module': 'Timestamp',
 'call_args': [{'name': 'now', 'type': 'Moment', 'value': 1655687784009}],
 'call_hash': '0x9741ab6b11f8e857f9473af31ea267db7e325a03b44840aafff335b68b787b96'}